## Code for calculating SCC p-values 
Note, this code is for using with Python three. 

In [2]:
# Import the modules we need. 

import xarray as xr
import zarr as zr
import pandas as pd
import numpy as np

In [3]:
# Load in the draws data, which is a zarr dataset
ds = xr.open_zarr('/shares/gcp/outputs/energy/scc_uncertainty/energy_scc_fulluncertainty_2019-11-21_v1.zarr')
ds

<xarray.Dataset>
Dimensions:     (discrate: 4, pctile: 19, rcp: 2, simulation: 100000, var_type: 8)
Coordinates:
  * discrate    (discrate) float64 0.01 0.025 0.03 0.05
  * pctile      (pctile) float64 0.05 0.1 0.15 0.2 0.25 ... 0.8 0.85 0.9 0.95
  * rcp         (rcp) object 'rcp45' 'rcp85'
  * simulation  (simulation) int64 0 1 2 3 4 5 ... 99995 99996 99997 99998 99999
  * var_type    (var_type) object 'mergeetl60' 'price0' ... 'witchglobiom42'
Data variables:
    scc         (discrate, simulation, rcp, var_type, pctile) float64 dask.array<chunksize=(1, 25000, 1, 2, 5), meta=np.ndarray>
Attributes:
    climate_output_version:     v2.1, 2019-06-21
    climate_parameter_author:   Kelly McCusker
    tag:                        energy-submission-2019-11-21
    climate_parameter_version:  2019-02-20
    damage_function_author:     Tamma Carleton
    damage_function:            energy_damage_coefficients_TINV_clim_income_s...
    contact:                    dgergel@rhg.com
    author:      

In [4]:
def get_p_val(var_type, discrate, rcp, ds):
	
	filtered = ds.sel(var_type = var_type).sel(discrate = discrate).sel(rcp = rcp)

	# Turn it into a dataframe
	df = filtered.to_dataframe().reset_index() 

	# Weight the scc values by their quantile
	df['reps'] = np.where( np.logical_or(df['pctile'] == 0.05,  df['pctile'] == 0.95) , 3, 2)
	df = df.loc[np.repeat(df.index.values, df['reps'] )]

	# Since the scc is below zero, we are going to find the number of draws above zero: 
	df['test_fail'] = np.where(df['scc']>0, 1, 0)

	number_failed = df.test_fail.sum()
	total = df.test_fail.count()

	p = number_failed * 2 / total

	print(p)
	return p

In [5]:
get_p_val(var_type = 'price014', discrate = 0.03, rcp = 'rcp85', ds = ds)

0.041968


0.041968

In [6]:
get_p_val(var_type = 'price014', discrate = 0.03, rcp = 'rcp45', ds = ds)

0.0110375


0.0110375

# I don't trust the above output, so am going to plot the values for rcp 45

In [7]:
df = ds.sel(var_type = 'price014').sel(discrate = 0.03).sel(rcp = 'rcp45') \
            .to_dataframe().reset_index()
df['reps'] = np.where( np.logical_or(df['pctile'] == 0.05,  df['pctile'] == 0.95) , 3, 2)
df = df.loc[np.repeat(df.index.values, df['reps'] )]
df.head()

,pctile,simulation,discrate,rcp,scc,var_type,reps
0,0.05,0,0.03,rcp45,-7.858727,price014,3
0,0.05,0,0.03,rcp45,-7.858727,price014,3
0,0.05,0,0.03,rcp45,-7.858727,price014,3
1,0.05,1,0.03,rcp45,-0.844233,price014,3
1,0.05,1,0.03,rcp45,-0.844233,price014,3
